## 使用xgboost分析文本情感
- 输入数据：使用` gensim.models.doc2vec.Doc2Vec(dm=0,dbow_words=1,size=80,min_count=2,iter=55,workers=cores,window=8)`构建PV_DBOW_model模型生成的文本向量
- 模型：使用xgb.sklearn.XGBClassifier。XGBoost Library疑因参数/训练数据数值向下溢出导致内存报错，因此本文没有直接使用xgboost库来构建模型。

In [11]:
#计算均方根误差RMSE
from __future__ import division
def calc_rmse(forecast,real):
    diff = (forecast-real)**2
    rmse = np.sqrt(diff.sum()/len(forecast))
    return rmse    

In [2]:
import xgboost as xgb
import numpy as np 
import pandas as pd
import sklearn
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV

In [3]:
xgb1 = xgb.sklearn.XGBClassifier(
 learning_rate =0.5,
 n_estimators=50,
 max_depth=10,
 min_child_weight=0.01,
 gamma=10.368977976003487,
 subsample=0.01,
 colsample_bytree=0.01,
 objective='multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 reg_lambda=0.01,
 reg_alpha=0.01,
 seed=27)  

In [4]:
pd_train = pd.read_csv('./data/pd_train.csv')
pd_test = pd.read_csv('./data/pd_test.csv')
pd_predict = pd.read_csv('./data/pd_predict.csv')

In [5]:
#训练xgboost模型
def modelfit1(alg, train_data_features, train_labels,predict_data_features,useTrainCV=True, cv_folds=5):

    if useTrainCV:
        params=alg.get_xgb_params()
        xgb_param=dict([(key,[params[key]]) for key in params])

        boost = xgb.sklearn.XGBClassifier()
        cvresult = GridSearchCV(boost,xgb_param,cv=cv_folds)
        cvresult.fit(train_data_features, train_labels)
        alg=cvresult.best_estimator_

    #Fit the algorithm on the data
    alg.fit(train_data_features, train_labels)

    #Predict training set:
    dtrain_predictions = alg.predict(predict_data_features)
    #dtrain_predprob = alg.predict_proba(train_data_features)[:,1]
    return dtrain_predictions

In [ ]:
import sklearn
# 训练模型，打印正确率
def modelfit(alg, train_data_features, train_labels,useTrainCV=True, cv_folds=5):

    if useTrainCV:
        params=alg.get_xgb_params()
        xgb_param=dict([(key,[params[key]]) for key in params])

        boost = xgb.sklearn.XGBClassifier()
        cvresult = GridSearchCV(boost,xgb_param,cv=cv_folds)
        cvresult.fit(train_data_features, train_labels)
        alg=cvresult.best_estimator_


    #Fit the algorithm on the data
    alg.fit(train_data_features, train_labels)

    #Predict training set:
    dtrain_predictions = alg.predict(train_data_features)
    dtrain_predprob = alg.predict_proba(train_data_features)[:,1]
    
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % sklearn.metrics.accuracy_score(train_labels, dtrain_predictions))

In [6]:
feature = [str(i) for i in range(80)]
%time test = modelfit1(xgb1,pd_train[feature],pd_train['score'],pd_test[feature])

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

CPU times: user 1min 56s, sys: 256 ms, total: 1min 56s
Wall time: 33.2 s


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
#计算均方根误差RMSE
from __future__ import division
def calc_rmse(forecast,real):
    diff = (forecast-real)**2
    rmse = np.sqrt(diff.sum()/len(forecast))
    return rmse

In [8]:
rmse = calc_rmse(test,pd_test.score)

In [9]:
score = 1/(1+rmse)

In [10]:
score

0.4613807295412556